# An Analysis of Movie Performance

In this part, you’ll gather data about popular movies and award winners. The goal is to build a dataset that you’ll later use to analyze what makes a movie successful and how awards and box office performance relate to one another.

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json
import time

### Part 1: Data Gathering
1. Scrape Best Picture Data.  
    * Scrape the [Best Picture wikipedia page](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture).  
    * Extract for each Movie:  
        * Year  
        * Film Title  
        * Winner (Yes/No)  
    * Data cleaning tips:  
        * Ensure that year and film title columns are clean and consistent (no footnotes, parentheses, etc.).
        * Save the results as best_picture.csv.  

In [2]:
URL = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture'

headers = {
    "User-Agent": "Movie_Agent"
}

resp = requests.get(URL, headers=headers)
soup = BeautifulSoup(resp.text)
#soup.prettify()

In [3]:
soup = BeautifulSoup(resp.text)
#soup.prettify()

In [4]:
winners = soup.findAll('tr', attrs={'style' : 'background:#FAEB86'})
winning_titles = [winner.td.text.strip() for winner in winners]

In [5]:
# Get all wikitables
all_tables = soup.findAll('table', attrs={'class' : 'wikitable'})

# Find just the tables with movie data by filtering on the 'Year of Film Release' column header
# This excludes the 2 wikitables at the very bottom of the page ('Age superlatives' and 'Production companies and distributors with multiple nominations and wins')
movie_tables = [table for table in all_tables if 'Year of Film Release' in table.find('tr').text]

In [6]:
# Create empty list to store dictionaries of movie data, ex. [{'Title': 'Wings', 'Movie_Year': '1927', 'Awards_Year': '1928', 'Winner':'True'}]
movie_info = []

# Iterate through all tables to extract movie data
for table in movie_tables:
    
    # Find all 'tr' tags ('table row'), skipping the first one since it just contains column headers
    rows = table.findAll('tr')[1:]
    
    # Iterate through all rows of the table to find year, movie, and winner status
    for row in rows:
        
        # If the row contains a 'th' ('table header') tag, extract the year and store it in a variable
        if len(row.findAll('th')) != 0:
            awards_year = row.th.a.text

            # If the awards_year contains a slash, only grab the later year
            if '/' in awards_year:
                awards_year = awards_year[:2] + awards_year[-2:]
        
        # Get the movie title in this row, if there is one
        if len(row.findAll('td')) != 0:
            title = row.td.text.strip()
            
            # Get the winner status by seeing if the title is in the winning_titles list
            if title in winning_titles:
                winner='Yes'
            else:
                winner='No'
        else:
            title = ''
        
        # If this row has a movie title, append the movie info to the movie_info list
        if title != '':
            movie_info_dict = {'Title': title, 'Awards_Year': awards_year, 'Winner': winner}
            movie_info.append(movie_info_dict)

In [7]:
# Find all italicized <i> tags inside wikitable elements
tables = soup.select("table.wikitable i")

titles = []

for i_tag in tables:
    # Extract only linked movie titles 
    link = i_tag.find("a")
    if link:
        title = link.get_text(strip=True)
        if title and title not in titles:
            titles.append(title)

print(f"Found {len(titles)} movie titles.\n")
for t in titles:
    print(t)


Found 602 movie titles.

Wings
7th Heaven
The Racket
The Broadway Melody
Alibi
The Hollywood Revue
In Old Arizona
The Patriot
All Quiet on the Western Front
The Big House
Disraeli
The Divorcee
The Love Parade
Cimarron
East Lynne
The Front Page
Skippy
Trader Horn
Grand Hotel
Arrowsmith
Bad Girl
The Champ
Five Star Final
One Hour with You
Shanghai Express
The Smiling Lieutenant
Cavalcade
42nd Street
A Farewell to Arms
I Am a Fugitive from a Chain Gang
Lady for a Day
Little Women
The Private Life of Henry VIII
She Done Him Wrong
Smilin' Through
State Fair
It Happened One Night
The Barretts of Wimpole Street
Cleopatra
Flirtation Walk
The Gay Divorcee
Here Comes the Navy
The House of Rothschild
Imitation of Life
One Night of Love
The Thin Man
Viva Villa!
The White Parade
Mutiny on the Bounty
Alice Adams
Broadway Melody of 1936
Captain Blood
David Copperfield
The Informer
The Lives of a Bengal Lancer
A Midsummer Night's Dream
Les Misérables
Naughty Marietta
Ruggles of Red Gap
Top Hat
The Gre

In [8]:
# Convert movie_info to a pandas DataFrame
movie_info_df = pd.DataFrame(movie_info)
movie_info_df

,Title,Awards_Year,Winner
0,Wings,1928,Yes
1,7th Heaven,1928,No
2,The Racket,1928,No
3,The Broadway Melody,1929,Yes
4,Alibi,1929,No
...,...,...,...
606,Emilia Pérez,2024,No
607,I'm Still Here,2024,No
608,Nickel Boys,2024,No
609,The Substance,2024,No


2. Gather Movie Data via TMDB API  
    a. Set up the API    
    * Create a free [TMDB account](https://developer.themoviedb.org/docs/getting-started)  
    * Generate an API key are review their documentation, especially:  
        * /discover/movie  
        * /movie/{movie_id}  
        * /search/movie  
    b. Collect top movies (2015-2024)  
    For each year from 2015 to 2024:  
        * Query TMDB for the top 100 movies (by vote count).  
        * For each movie, gather:  
            * Title  
            * Release Year  
            * Genre(s)  
            * Vote Average  
            * Vote Count  
            * Budget  
            * Revenue  
            * TMDB ID  
        * Store all results in a single DataFrame and export to movies_2015_2024.csv.
        * Hint: TMDB rate limits are generous for free accounts, but you should pause between requests (eg. time.sleep(0.25)). 
        * Some Oscar films may not appear in the top 100 by vote count. For any missing, use the /search/movie endpoint to add it.  


In [9]:
with open('api_movies.json') as fi:
    credentials = json.load(fi)
    
key = credentials['key']

In [10]:
endpoint = 'https://api.themoviedb.org/3'

In [11]:
endpoint = "https://api.themoviedb.org/3/discover/movie"

movies = []

for year in range (2015, 2025):
    for page in range (1,6):
        params = {
            'primary_release_year': year,
            'sort_by' : 'vote_count.desc',
            'api_key' : key
        }


        response = requests.get(endpoint, params = params)
        res = response.json()['results']
        
        for movie in res:
            movies.append(movie)
            
        
        time.sleep(0.25)

In [14]:
print(movies)

[{'adult': False, 'backdrop_path': '/kIBK5SKwgqIIuRKhhWrJn3XkbPq.jpg', 'genre_ids': [28, 12, 878], 'id': 99861, 'original_language': 'en', 'original_title': 'Avengers: Age of Ultron', 'overview': 'When Tony Stark tries to jumpstart a dormant peacekeeping program, things go awry and Earth’s Mightiest Heroes are put to the ultimate test as the fate of the planet hangs in the balance. As the villainous Ultron emerges, it is up to The Avengers to stop him from enacting his terrible plans, and soon uneasy alliances and unexpected action pave the way for an epic and unique global adventure.', 'popularity': 14.9911, 'poster_path': '/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg', 'release_date': '2015-04-22', 'title': 'Avengers: Age of Ultron', 'video': False, 'vote_average': 7.271, 'vote_count': 23837}, {'adult': False, 'backdrop_path': '/gqrnQA6Xppdl8vIb2eJc58VC1tW.jpg', 'genre_ids': [28, 12, 878], 'id': 76341, 'original_language': 'en', 'original_title': 'Mad Max: Fury Road', 'overview': 'An apocalyptic

In [33]:
movie_titles = []
for movie in movies:
    movie_titles.append(movie['title'])  

In [35]:
import datetime

release_year = []
for movie in movies:
    release_year.append(movie['release_date'])  

    years = [datetime.datetime.strptime(date_str, "%Y-%m-%d").year for date_str in release_year]


In [38]:
genre_ids = []
for movie in movies:
    genre_ids.append(movie['id'])  
    
print(genre_ids)

[99861, 76341, 150540, 135397, 286217, 102899, 140607, 281957, 207703, 273248, 264660, 131634, 216015, 168259, 206647, 211672, 294254, 262500, 264644, 318846, 99861, 76341, 150540, 135397, 286217, 102899, 140607, 281957, 207703, 273248, 264660, 131634, 216015, 168259, 206647, 211672, 294254, 262500, 264644, 318846, 99861, 76341, 150540, 135397, 286217, 102899, 140607, 281957, 207703, 273248, 264660, 131634, 216015, 168259, 206647, 211672, 294254, 262500, 264644, 318846, 99861, 76341, 150540, 135397, 286217, 102899, 140607, 281957, 207703, 273248, 264660, 131634, 216015, 168259, 206647, 211672, 294254, 262500, 264644, 318846, 99861, 76341, 150540, 135397, 286217, 102899, 140607, 281957, 207703, 273248, 264660, 131634, 216015, 168259, 206647, 211672, 294254, 262500, 264644, 318846, 293660, 271110, 284052, 297761, 259316, 329865, 209112, 313369, 269149, 330459, 324786, 274870, 246655, 277834, 296096, 127380, 372058, 291805, 283366, 381284, 293660, 271110, 284052, 297761, 259316, 329865, 2

**Optional Extension: Actors and Actresses** 

1. Scrape Wikipedia for Best Actor and Best Actress Data
    * Scrape the following Wikipedia pages:  
        * [Best Actor](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor)
        * [Best Actress](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress)
    * Each apge contains tables of winners and nominees by year.
    * Extract the following columns:  
        * Year
        * Actor/Actress Name
        * Film Title
        * Winner (Yes/No)
    * Data cleaning tips:  
        * Remove footnote markers from names and movie titles.
        * Ensure that you save just the release year (eg. 2009 instead of 2009 (82nd))
        * Store the cleaned data as two csv files:  
            * best_actor.csv
            * best_actress.csv  

2. Collect Actor and Actress Filmographies  
    Using the data from your actor and actresses CSVs:  
    * Search TMDB for each recent performer (using /search/person). Note: you can start with 2015-2024 initially, but, if time allows, you can go back even further.
    * For each person, retrieve their movie credits using /person/{person_id}/movie_credits.  
    * Extract relevant fields for each movie, such as:  
        * Actor/Actress Name  
        * Movie Title  
        * Character Name (optional)  
        * Release Year  
        * Movie ID
    * Combine all filmographies into one file, actor_filmography.csv